# Satellite elevation-azimuth diagrams
### Guillaume Witz, Science IT Support, Bern University

In [1]:
#import packages
import os, re, glob
from datetime import date, timedelta

import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
from bz2 import BZ2File as bzopen

import ipywidgets as ipw
from ipywidgets import interact, interactive, fixed, interact_manual

import aiub

In [2]:
#user paramters

#file location
address = 'http://ftp.aiub.unibe.ch/BSWUSER52/STA/2019/'
#address_sat = 'http://ftp.aiub.unibe.ch/CODE/'
address_sat = 'http://ftp.aiub.unibe.ch/CODE_MGEX/CODE/'
#station coordinates file
coord_file = 'COD19001.CRD.Z'
#station to observe
station_name = '14001M004'

In [5]:
#import station data
stations = aiub.import_stations(address,coord_file)

In [6]:
#select station and calculate ellipsoidal coordinates
curr_stat = stations[stations.statname == station_name].iloc[0]
rad_stat_coord = aiub.cartesian_to_ellipsoidal(curr_stat.x_pos, curr_stat.y_pos,curr_stat.z_pos)
curr_stat['rad_stat_coord'] = rad_stat_coord

In [6]:
date_min, date_max = aiub.date_picker()

In [7]:
#create a list of dates between start and end date 
date_list = [date_min.value + timedelta(days=x) for x in range((date_max.value-date_min.value).days+1)]
#calculate gps weeks times (year, week, day)
gps_weeks = [aiub.date_to_gpsweeks(x) for x in date_list]

#load satellite data
temp_pd = pd.concat([aiub.import_RM_file(address_sat, g) for g in gps_weeks]).reset_index()

#reanme satellite types
sat_dict = {'G':'GPS','R':'GLONASS','E':'Galileo','C':'BeiDou','J':'QZSS'}
temp_pd['satellite'] = temp_pd.satellite.apply(lambda x : sat_dict[x])

In [8]:
#caluclate elevation and azimuth for all satellites for given station
temp_pd = aiub.el_al_single_station_fast(temp_pd, curr_stat)
temp_pd['curr_stat'] = curr_stat.statname

In [9]:
aiub.interactive_rad(temp_pd, stations)

Output()